
<H3 align='center'> An Attention-Based Architecture for
Hierarchical Classification with CNNs </H3>

<H5 align='center'> CIFAR-10 </H3>

<hr style="height:2px;border:none"/>


# Dependencies

In [ ]:
import keras
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate, Add, Softmax
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.initializers import he_normal
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard, CSVLogger
from keras.utils.data_utils import get_file
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from keras.utils.vis_utils import plot_model
import pickle
from keras.models import load_model
from keras.callbacks import CSVLogger
from tensorflow.keras.utils import set_random_seed
from scipy import stats

# Computes hierarchical metrics proposed by Kiritchenko et al (2005)
def hierarchical_metrics(true,pred):
  true_labels = []
  true_fine = true[2].argmax(axis=1)
  true_c2 = true[1].argmax(axis=1)
  true_c1 = true[0].argmax(axis=1)
  for i in range(len(true_fine)):
    true_labels.append([true_c1[i],true_c2[i],true_fine[i]])
  pred_labels = []
  pred_c1 = pred[0].argmax(axis = 1)
  pred_c2 = pred[1].argmax(axis = 1)
  pred_fine = pred[2].argmax(axis = 1) 
  for i in range(len(pred_c1)):
    pred_labels.append([pred_c1[i],pred_c2[i],pred_fine[i]])
  preci = precision(true_labels,pred_labels)
  reca = recall(true_labels,pred_labels)
  f_1 = f1(true_labels,pred_labels)
  return preci,reca,f_1

# Hierarchical metrics, proposed by Kiritchenko et al (2005)
# Implementation 
# https://gitlab.com/dacs-hpi/hiclass/-/blob/main/hiclass/metrics.py


def precision(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute precision score for hierarchical classification.

    hP = sum(|S intersection T|) / sum(|S|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    precision : float
        What proportion of positive identifications was actually correct?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(prediction)
        )
    precision = sum_intersection / sum_prediction_and_ancestors
    return precision


def recall(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute recall score for hierarchical classification.

    hR = sum(|S intersection T|) / sum(|T|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    recall : float
        What proportion of actual positives was identified correctly?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(ground_truth)
        )
    recall = sum_intersection / sum_prediction_and_ancestors
    return recall


def f1(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute f1 score for hierarchical classification.

    hF = 2 * hP * hR / (hP + hR),
    where hP is the hierarchical precision and hR is the hierarchical recall.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    f1 : float
        Weighted average of the precision and recall
    """
    assert len(y_true) == len(y_pred)
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    f1 = 2 * prec * rec / (prec + rec)
    return f1

# General Settings

In [ ]:
#-------- dimensions ---------
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 3)
#-----------------------------

train_size = 50000

#--- coarse 1 classes ---
num_c_1 = 2
#--- coarse 2 classes ---
num_c_2 = 7
#--- fine classes ---
num_classes  = 10

batch_size   = 128
epochs       = 60  

In [ ]:
#-------------------- data loading ----------------------
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, num_classes)
y_cm = y_test
y_test = to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#---------------- data preprocessing -------------------
x_train = (x_train-np.mean(x_train)) / np.std(x_train)
x_test = (x_test-np.mean(x_test)) / np.std(x_test)

In [ ]:
#---------------------- make coarse 2 labels --------------------------
parent_f = {
  2:3, 3:5, 5:5,
  1:2, 7:6, 4:6,
  0:0, 6:4, 8:1, 9:2
}

y_c2_train = np.zeros((y_train.shape[0], num_c_2)).astype("float32")
y_c2_test = np.zeros((y_test.shape[0], num_c_2)).astype("float32")
for i in range(y_c2_train.shape[0]):
  y_c2_train[i][parent_f[np.argmax(y_train[i])]] = 1.0
for i in range(y_c2_test.shape[0]):
  y_c2_test[i][parent_f[np.argmax(y_test[i])]] = 1.0

#---------------------- make coarse 1 labels --------------------------
parent_c2 = {
  0:0, 1:0, 2:0,
  3:1, 4:1, 5:1, 6:1
}
y_c1_train = np.zeros((y_c2_train.shape[0], num_c_1)).astype("float32")
y_c1_test = np.zeros((y_c2_test.shape[0], num_c_1)).astype("float32")
for i in range(y_c1_train.shape[0]):
  y_c1_train[i][parent_c2[np.argmax(y_c2_train[i])]] = 1.0
for i in range(y_c1_test.shape[0]):
  y_c1_test[i][parent_c2[np.argmax(y_c2_test[i])]] = 1.0

In [ ]:
# Learning rate scheduler
def scheduler(epoch):
  learning_rate_init = 0.003
  if epoch > 42:
    learning_rate_init = 0.0005
  if epoch > 52:
    learning_rate_init = 0.0001
  return learning_rate_init

In [ ]:
# Loss Weights modifier, when BT-strategy is used
class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 10:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 20:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 30:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)

#Flat CNN Base B 

In [ ]:
#----------------------- model definition ---------------------------
img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, fine_pred, name='flat_cnn_base_b')

#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              metrics=['accuracy'])


In [ ]:
change_lr = LearningRateScheduler(scheduler)

# Training
history_base_b = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=change_lr,
          validation_data=(x_test, y_test))

# Evaluation on test set
score_base_b = model.evaluate(x_test, y_test, verbose=0)
parameters_base_b = np.sum([K.count_params(w) for w in model.trainable_weights])

# Results
print("--- Flat CNN Base B ---")
print("Accuracy:",score_base_b[1])
print("Parameters:","{:,}".format(parameters_base_b))

--- Flat CNN Base B ---
Accuracy: 0.8210999965667725
Parameters: 7,851,338


# B-CNN Base B

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  

img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(c_1_bch)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(512, activation='relu', name='c2_fc_cifar10_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(512, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(c_2_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(1024, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bcnn_base_b')

#----------------------- compile  ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

Model: "bcnn_base_b"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_10 (BatchN  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 ormalization)                                                                                    
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_1

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_b_cnn_b = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_b_cnn_b = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_b_cnn_b,reca_b_cnn_b,f1_b_cnn_b= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- B-CNN Base B ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_b_cnn_b[4])
print("Accuracy level 2:",score_b_cnn_b[5])
print("Accuracy level 3:",score_b_cnn_b[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_b_cnn_b)
print("Recall:",reca_b_cnn_b)
print("f1:",f1_b_cnn_b)
print("Parameters:","{:,}".format(parameters_b_cnn_b))

--- B-CNN Base B ---
--- Accuracy per level ---
Accuracy level 1: 0.9581000208854675
Accuracy level 2: 0.8691999912261963
Accuracy level 3: 0.8438000082969666
--- Hierarchical Metrics ---
Precision: 0.9080192165558019
Recall: 0.9100370370370371
f1: 0.9090270070292268
Parameters: 12,382,035


# BA-CNN Base B 

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

# neurons of all dense layers on each branch 
branch_neurons = 256

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  

img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(branch_neurons, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(branch_neurons, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch_out = Dropout(0.5)(c_1_bch)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(branch_neurons, activation='relu', name='c2_fc_cifar10_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(branch_neurons, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch_out = Dropout(0.5)(c_2_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(branch_neurons, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(branch_neurons, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x_out = Dropout(0.5)(x)


#-- Att for coarse 1---
# Coarse 1
sfcn_1_1 = Dense(64, name='fc1_1')(c_1_bch_out)
sfcn_1_1 = Dense(1, name='fc1_2')(sfcn_1_1)
# Coarse 2
sfcn_1_2 = Dense(64, name='fc1_3')(c_2_bch_out)
sfcn_1_2 = Dense(1, name='fc1_4')(sfcn_1_2)
# Fine
sfcn_1_3 = Dense(64, name='fc1_5')(x_out)
sfcn_1_3 = Dense(1, name='fc1_6')(sfcn_1_3)

score_vector_1 = Concatenate()([sfcn_1_1,sfcn_1_2,sfcn_1_3]) # Score vector 1
att_weights_1 = Activation('softmax', name='attention_weights_1')(score_vector_1) # Attention weights 1
weightned_sum_1 = Add()([c_1_bch_out*att_weights_1[0][0],c_2_bch_out*att_weights_1[0][1],x_out*att_weights_1[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_1_concat = Concatenate()([c_1_bch_out,weightned_sum_1])
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(coarse_1_concat)


#-- Att for coarse 2---

# Coarse 1
sfcn_2_1 = Dense(64, name='fc2_1')(c_1_bch_out)
sfcn_2_1 = Dense(1, name='fc2_2')(sfcn_2_1)
# Coarse 2
sfcn_2_2 = Dense(64, name='fc2_3')(c_2_bch_out)
sfcn_2_2 = Dense(1, name='fc2_4')(sfcn_2_2)
# Fine
sfcn_2_3 = Dense(64, name='fc2_5')(x_out)
sfcn_2_3 = Dense(1, name='fc2_6')(sfcn_2_3)

score_vector_2 = Concatenate()([sfcn_2_1,sfcn_2_2,sfcn_2_3]) # Score vector 1
att_weights_2 = Activation('softmax', name='attention_weights_2')(score_vector_2) # Attention weights 1
weightned_sum_2 = Add()([c_1_bch_out*att_weights_2[0][0],c_2_bch_out*att_weights_2[0][1],x_out*att_weights_2[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_2_concat = Concatenate()([c_2_bch_out,weightned_sum_2])
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(coarse_2_concat)


#-- Att for fine---

# Coarse 1
sfcn_3_1 = Dense(64, name='fc3_1')(c_1_bch_out)
sfcn_3_1 = Dense(1, name='fc3_2')(sfcn_3_1)
# Coarse 2
sfcn_3_2 = Dense(64, name='fc3_3')(c_2_bch_out)
sfcn_3_2 = Dense(1, name='fc3_4')(sfcn_3_2)
# Fine
sfcn_3_3 = Dense(64, name='fc3_5')(x_out)
sfcn_3_3 = Dense(1, name='fc3_6')(sfcn_3_3)

score_vector_3 = Concatenate()([sfcn_3_1,sfcn_3_2,sfcn_3_3]) # Score vector 1
att_weights_3 = Activation('softmax', name='attention_weights_3')(score_vector_3) # Attention weights 1
weightned_sum_3 = Add()([c_1_bch_out*att_weights_3[0][0],c_2_bch_out*att_weights_3[0][1],x_out*att_weights_3[0][2]]) # Weightned sum 3

# Concat and prediction
fine_concat = Concatenate()([x_out,weightned_sum_3])
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(fine_concat)

model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bacnn_base_b')

#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

Model: "bacnn_base_b"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_24 (BatchN  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 ormalization)                                                                                    
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bacnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_ba_cnn_b = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_ba_cnn_b = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_ba_cnn_b,reca_ba_cnn_b,f1_ba_cnn_b= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- BA-CNN Base B ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_ba_cnn_b[4])
print("Accuracy level 2:",score_ba_cnn_b[5])
print("Accuracy level 3:",score_ba_cnn_b[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_ba_cnn_b)
print("Recall:",reca_ba_cnn_b)
print("f1:",f1_ba_cnn_b)
print("Parameters:","{:,}".format(parameters_ba_cnn_b))

--- BA-CNN Base B ---
--- Accuracy per level ---
Accuracy level 1: 0.9678999781608582
Accuracy level 2: 0.8777999877929688
Accuracy level 3: 0.8420000076293945
--- Hierarchical Metrics ---
Precision: 0.9141343018084791
Recall: 0.9135925925925926
f1: 0.913863366923533
Parameters: 8,718,812


# H-CNN Base B

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  

img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch_flatt = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch_flatt)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(c_1_bch)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch_flatt = Flatten(name='c2_flatten')(x)
c_2_bch_concat = Concatenate()([c_1_bch_flatt,c_2_bch_flatt]) # Conectivity Pattern
c_2_bch = Dense(512, activation='relu', name='c2_fc_cifar100_1')(c_2_bch_concat)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(512, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(c_2_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- fine block ---
x_flatt = Flatten(name='flatten')(x)
x = Concatenate()([c_2_bch_concat,x_flatt]) # Conectivity Pattern
x = Dense(1024, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='hcnn_base_b')

#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

Model: "hcnn_base_b"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_38 (BatchN  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 ormalization)                                                                                    
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_3

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_hcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_h_cnn_b = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_h_cnn_b = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_h_cnn_b,reca_h_cnn_b,f1_h_cnn_b= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- H-CNN Base B ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_h_cnn_b[4])
print("Accuracy level 2:",score_h_cnn_b[5])
print("Accuracy level 3:",score_h_cnn_b[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_h_cnn_b)
print("Recall:",reca_h_cnn_b)
print("f1:",f1_h_cnn_b)
print("Parameters:","{:,}".format(parameters_h_cnn_b))

--- H-CNN Base B ---
--- Accuracy per level ---
Accuracy level 1: 0.9585999846458435
Accuracy level 2: 0.8716999888420105
Accuracy level 3: 0.836899995803833
--- Hierarchical Metrics ---
Precision: 0.906301854788156
Recall: 0.9102962962962963
f1: 0.9082946839372494
Parameters: 29,159,251


# Flat CNN Base C 

In [ ]:
#----------get VGG16 pre-trained weights--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')
#----------------------- model definition ---------------------------
img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- block 5 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = BatchNormalization()(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc_cifar100_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc_cifar100_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, fine_pred, name='flat_cnn_base_c')
model.load_weights(weights_path, by_name=True)

#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

553476096/553467096 [==============================] - 87s 0us/step
Model: "flat_cnn_base_c"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_52 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_53 (Bat  (None, 32, 32, 64)       256       
 chNormalization)                                                
                                                 

In [ ]:
change_lr = LearningRateScheduler(scheduler)

# Training
history_base_c = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=change_lr,
          validation_data=(x_test, y_test))

# Evaluation on test set
score_base_c = model.evaluate(x_test, y_test, verbose=0)
parameters_base_c = np.sum([K.count_params(w) for w in model.trainable_weights])

# B-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------get VGG16 pre-trained weights--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')

#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  


img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(512, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(512, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(c_1_bch)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(1024, activation='relu', name='c2_fc_cifar10_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(c_2_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- block 5 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = BatchNormalization()(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bcnn_base_c')
model.load_weights(weights_path, by_name=True)
#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

Model: "bcnn_base_c"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_67 (BatchN  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 ormalization)                                                                                    
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_6

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_b_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_b_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_b_cnn_c,reca_b_cnn_c,f1_b_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- B-CNN Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_b_cnn_c[4])
print("Accuracy level 2:",score_b_cnn_c[5])
print("Accuracy level 3:",score_b_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_b_cnn_c)
print("Recall:",reca_b_cnn_c)
print("f1:",f1_b_cnn_c)
print("Parameters:","{:,}".format(parameters_b_cnn_c))

--- B-CNN Base C ---
--- Accuracy per level ---
Accuracy level 1: 0.9603000283241272
Accuracy level 2: 0.9053000211715698
Accuracy level 3: 0.8884999752044678
--- Hierarchical Metrics ---
Precision: 0.9315478389360916
Recall: 0.933962962962963
f1: 0.9327538376179028
Parameters: 49,671,251


# BA-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

# neurons of all dense layers on each branch 
branch_neurons = 256

In [ ]:
#----------get VGG16 pre-trained weights--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')

#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  


img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(branch_neurons, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(branch_neurons, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch_out = Dropout(0.5)(c_1_bch)
#c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(c_1_bch_out)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(branch_neurons, activation='relu', name='c2_fc_cifar10_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(branch_neurons, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch_out = Dropout(0.5)(c_2_bch)
#c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(c_2_bch_out)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- block 5 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = BatchNormalization()(x)

#--- fine block ---
x = Flatten(name='flatten')(x)
x = Dense(branch_neurons, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(branch_neurons, activation='relu', name='fc_cifar10_2')(x)
x = BatchNormalization()(x)
x_out = Dropout(0.5)(x)

#-- Att for coarse 1---
# Coarse 1
sfcn_1_1 = Dense(64, name='fc1_1')(c_1_bch_out)
sfcn_1_1 = Dense(1, name='fc1_2')(sfcn_1_1)
# Coarse 2
sfcn_1_2 = Dense(64, name='fc1_3')(c_2_bch_out)
sfcn_1_2 = Dense(1, name='fc1_4')(sfcn_1_2)
# Fine
sfcn_1_3 = Dense(64, name='fc1_5')(x_out)
sfcn_1_3 = Dense(1, name='fc1_6')(sfcn_1_3)

score_vector_1 = Concatenate()([sfcn_1_1,sfcn_1_2,sfcn_1_3]) # Score vector 1
att_weights_1 = Activation('softmax', name='attention_weights_1')(score_vector_1) # Attention weights 1
weightned_sum_1 = Add()([c_1_bch_out*att_weights_1[0][0],c_2_bch_out*att_weights_1[0][1],x_out*att_weights_1[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_1_concat = Concatenate()([c_1_bch_out,weightned_sum_1])
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(coarse_1_concat)


#-- Att for coarse 2---

# Coarse 1
sfcn_2_1 = Dense(64, name='fc2_1')(c_1_bch_out)
sfcn_2_1 = Dense(1, name='fc2_2')(sfcn_2_1)
# Coarse 2
sfcn_2_2 = Dense(64, name='fc2_3')(c_2_bch_out)
sfcn_2_2 = Dense(1, name='fc2_4')(sfcn_2_2)
# Fine
sfcn_2_3 = Dense(64, name='fc2_5')(x_out)
sfcn_2_3 = Dense(1, name='fc2_6')(sfcn_2_3)

score_vector_2 = Concatenate()([sfcn_2_1,sfcn_2_2,sfcn_2_3]) # Score vector 1
att_weights_2 = Activation('softmax', name='attention_weights_2')(score_vector_2) # Attention weights 1
weightned_sum_2 = Add()([c_1_bch_out*att_weights_2[0][0],c_2_bch_out*att_weights_2[0][1],x_out*att_weights_2[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_2_concat = Concatenate()([c_2_bch_out,weightned_sum_2])
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(coarse_2_concat)


#-- Att for fine---

# Coarse 1
sfcn_3_1 = Dense(64, name='fc3_1')(c_1_bch_out)
sfcn_3_1 = Dense(1, name='fc3_2')(sfcn_3_1)
# Coarse 2
sfcn_3_2 = Dense(64, name='fc3_3')(c_2_bch_out)
sfcn_3_2 = Dense(1, name='fc3_4')(sfcn_3_2)
# Fine
sfcn_3_3 = Dense(64, name='fc3_5')(x_out)
sfcn_3_3 = Dense(1, name='fc3_6')(sfcn_3_3)

score_vector_3 = Concatenate()([sfcn_3_1,sfcn_3_2,sfcn_3_3]) # Score vector 1
att_weights_3 = Activation('softmax', name='attention_weights_3')(score_vector_3) # Attention weights 1
weightned_sum_3 = Add()([c_1_bch_out*att_weights_3[0][0],c_2_bch_out*att_weights_3[0][1],x_out*att_weights_3[0][2]]) # Weightned sum 3

# Concat and prediction
fine_concat = Concatenate()([x_out,weightned_sum_3])
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(fine_concat)




model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bacnn_base_c')
model.load_weights(weights_path, by_name=True)
#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

Model: "bacnn_base_c"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_86 (BatchN  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 ormalization)                                                                                    
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_ba_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_ba_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_ba_cnn_c,reca_ba_cnn_c,f1_ba_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- BA-CNN Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_ba_cnn_c[4])
print("Accuracy level 2:",score_ba_cnn_c[5])
print("Accuracy level 3:",score_ba_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_ba_cnn_c)
print("Recall:",reca_ba_cnn_c)
print("f1:",f1_ba_cnn_c)
print("Parameters:","{:,}".format(parameters_ba_cnn_c))

--- BA-CNN Base C ---
--- Accuracy per level ---
Accuracy level 1: 0.9833999872207642
Accuracy level 2: 0.9225999712944031
Accuracy level 3: 0.888700008392334
--- Hierarchical Metrics ---
Precision: 0.9437250258990676
Recall: 0.9447037037037037
f1: 0.9442141112015991
Parameters: 18,752,732


# H-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------get VGG16 pre-trained weights--------
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')

#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  
img_input = Input(shape=input_shape, name='input')

#--- block 1 ---
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#--- block 2 ---
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#--- coarse 1 branch ---
c_1_bch_flatt = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch_flatt)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu', name='c1_fc2')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(num_c_1, activation='softmax', name='c1_predictions_cifar10')(c_1_bch)

#--- block 3 ---
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#--- coarse 2 branch ---
c_2_bch_flatt = Flatten(name='c2_flatten')(x)
c_2_bch_concat = Concatenate()([c_1_bch_flatt,c_2_bch_flatt]) # Conectivity Pattern
c_2_bch = Dense(1024, activation='relu', name='c2_fc_cifar100_1')(c_2_bch_concat)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu', name='c2_fc2')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(num_c_2, activation='softmax', name='c2_predictions_cifar10')(c_2_bch)

#--- block 4 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#--- block 5 ---
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = BatchNormalization()(x)

#--- fine block ---
x_flatt = Flatten(name='flatten')(x)
x = Concatenate()([c_2_bch_concat,x_flatt]) # Conectivity Pattern
x = Dense(4096, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar10')(x)

model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='hcnn_base_c')
model.load_weights(weights_path, by_name=True)
#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.003, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma],
              metrics=['accuracy'])
model.summary()

Model: "hcnn_base_c"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['input[0][0]']                  
                                                                                                  
 batch_normalization_105 (Batch  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization_1

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_h_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_h_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_h_cnn_c,reca_h_cnn_c,f1_h_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- H-CNN Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_h_cnn_c[4])
print("Accuracy level 2:",score_h_cnn_c[5])
print("Accuracy level 3:",score_h_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_h_cnn_c)
print("Recall:",reca_h_cnn_c)
print("f1:",f1_h_cnn_c)
print("Parameters:","{:,}".format(parameters_h_cnn_c))

--- H-CNN Base C ---
--- Accuracy per level ---
Accuracy level 1: 0.9592000246047974
Accuracy level 2: 0.9002000093460083
Accuracy level 3: 0.8792999982833862
--- Hierarchical Metrics ---
Precision: 0.9280221811460259
Recall: 0.9297407407407408
f1: 0.9288806660499537
Parameters: 106,095,699


# Summary

In [ ]:
summary = {'':['Flat CNN Base B','B-CNN Base B','BA-CNN Base B','H-CNN Base B'],'Coarse 1': [0,score_b_cnn_b[4],score_ba_cnn_b[4],score_h_cnn_b[4]],'Coarse 2': [0,score_b_cnn_b[5],score_ba_cnn_b[5],score_h_cnn_b[5]],'Fine': [score_base_b[1],score_b_cnn_b[6],score_ba_cnn_b[6],score_h_cnn_b[6]],'Precision':[0,preci_b_cnn_b,preci_ba_cnn_b,preci_h_cnn_b],'Recall':[0,reca_b_cnn_b,reca_ba_cnn_b,reca_h_cnn_b],'f1':[0,f1_b_cnn_b,f1_ba_cnn_b,f1_h_cnn_b],'Parameters': [parameters_base_b ,parameters_b_cnn_b,parameters_ba_cnn_b,parameters_h_cnn_b]}
summary = pd.DataFrame(summary)
summary['Parameters'] = (summary['Parameters'].astype(float)/1000000).round(2).astype(str) + 'MM'
summary = summary.set_index('')
summary.style.highlight_max()

,Coarse 1,Coarse 2,Fine,Precision,Recall,f1,Parameters
,,,,,,,
Flat CNN Base B,0.000000,0.000000,0.821100,0.000000,0.000000,0.000000,7.85MM
B-CNN Base B,0.958100,0.869200,0.843800,0.908019,0.910037,0.909027,12.38MM
BA-CNN Base B,0.967900,0.877800,0.842000,0.914134,0.913593,0.913863,8.72MM
H-CNN Base B,0.958600,0.871700,0.836900,0.906302,0.910296,0.908295,29.16MM


In [31]:
summary = {'':['Flat CNN Base C','B-CNN Base C','BA-CNN Base C','H-CNN Base C'],'Coarse 1': [0,score_b_cnn_c[4],score_ba_cnn_c[4],score_h_cnn_c[4]],'Coarse 2': [0,score_b_cnn_c[5],score_ba_cnn_c[5],score_h_cnn_c[5]],'Fine': [score_base_c[1],score_b_cnn_c[6],score_ba_cnn_c[6],score_h_cnn_c[6]],'Precision':[0,preci_b_cnn_c,preci_ba_cnn_c,preci_h_cnn_c],'Recall':[0,reca_b_cnn_c,reca_ba_cnn_c,reca_h_cnn_c],'f1':[0,f1_b_cnn_c,f1_ba_cnn_c,f1_h_cnn_c],'Parameters': [parameters_base_c ,parameters_b_cnn_c,parameters_ba_cnn_c,parameters_h_cnn_c]}
summary = pd.DataFrame(summary)
summary['Parameters'] = (summary['Parameters'].astype(float)/1000000).round(2).astype(str) + 'MM'
summary = summary.set_index('')
summary.style.highlight_max()

,Coarse 1,Coarse 2,Fine,Precision,Recall,f1,Parameters
,,,,,,,
Flat CNN Base C,0.000000,0.000000,0.884500,0.000000,0.000000,0.000000,39.95MM
B-CNN Base C,0.960300,0.905300,0.888500,0.931548,0.933963,0.932754,49.67MM
BA-CNN Base C,0.983400,0.922600,0.888700,0.943725,0.944704,0.944214,18.75MM
H-CNN Base C,0.959200,0.900200,0.879300,0.928022,0.929741,0.928881,106.1MM
